In [1]:
from helper import *
from Simulation.WaitTime import WaitTime
from Simulation.MoveJoint import MoveJoint
from Simulation.MoveLinear import MoveLinear
from Simulation.Simulation import Simulation
from neura_dual_quaternions import DualQuaternion, Quaternion

In [2]:
q2 = np.array([0, deg2rad(-60), 0, deg2rad(-60), 0, deg2rad(-60), 0])
q3 = -q2

q4 = np.array([deg2rad(-180), deg2rad(-90), deg2rad(-180), deg2rad(-150), deg2rad(-180), deg2rad(-120), deg2rad(-180)])
q5 = -q4

x1 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.9, np.array([0,1,0])), np.array([1,0,0.3]))
x2 = DualQuaternion.fromQuatPos(Quaternion.fromAxisAngle(np.pi*0.8, np.array([1,0,0])), np.array([1,0.5,0.5]))     
print(x1)
print(x2)
task_list = np.array([WaitTime(2), MoveJoint(q3, 3), MoveLinear(x1, 2), MoveLinear(x2, 2)])

DualQuaternion(Real: Quaternion(0.156, 0.000, 0.988, 0.000), Dual: Quaternion(0.000, -0.070, 0.000, 0.517))
DualQuaternion(Real: Quaternion(0.309, 0.951, 0.000, 0.000), Dual: Quaternion(-0.476, 0.155, 0.315, -0.161))


In [3]:
sim = Simulation(task_list)



In [4]:
sim.start()

Done!
